In [3]:
# Load libraries
import pandas as pd
import numpy as np

In [4]:
filepath = "/Users/brucechen/Documents/IDS720 PDS/pds-2025-opioids-study_on_opioids/arcos_all.zip"

# sample read to see the structure

sample = pd.read_csv(filepath, compression="zip", nrows=100, delimiter="\t")

sample.head()

,REPORTER_DEA_NO,REPORTER_BUS_ACT,REPORTER_NAME,REPORTER_ADDL_CO_INFO,REPORTER_ADDRESS1,REPORTER_ADDRESS2,REPORTER_CITY,REPORTER_STATE,REPORTER_ZIP,REPORTER_COUNTY,...,DRUG_NAME,Measure,MME_Conversion_Factor,Dosage_Strength,TRANSACTION_DATE,Combined_Labeler_Name,Reporter_family,CALC_BASE_WT_IN_GM,DOSAGE_UNIT,MME
0,RM0220688,DISTRIBUTOR,MCKESSON CORPORATION,NaN,DBA MCKESSON DRUG CO.,3000 KENSKILL AVE,WASHINGTON CT HOUSE,OH,43160,FAYETTE,...,CODEINE,TAB,0.15,30.0,2011-01-14,Teva,McKesson Corporation,2.21010,100.0,331.515
1,RM0220688,DISTRIBUTOR,MCKESSON CORPORATION,NaN,DBA MCKESSON DRUG CO.,3000 KENSKILL AVE,WASHINGTON CT HOUSE,OH,43160,FAYETTE,...,OXYCODONE,TAB,1.50,10.0,2011-02-08,"Par Pharmaceutical, Inc.",McKesson Corporation,0.89650,100.0,1344.750
2,RM0220688,DISTRIBUTOR,MCKESSON CORPORATION,NaN,DBA MCKESSON DRUG CO.,3000 KENSKILL AVE,WASHINGTON CT HOUSE,OH,43160,FAYETTE,...,HYDROCODONE,TAB,1.00,7.5,2011-03-07,SpecGx LLC,McKesson Corporation,0.45405,100.0,454.050
3,RM0220688,DISTRIBUTOR,MCKESSON CORPORATION,NaN,DBA MCKESSON DRUG CO.,3000 KENSKILL AVE,WASHINGTON CT HOUSE,OH,43160,FAYETTE,...,METHADONE,TAB,4.00,10.0,2011-03-01,SpecGx LLC,McKesson Corporation,3.57760,400.0,14310.400
4,RM0220688,DISTRIBUTOR,MCKESSON CORPORATION,NaN,DBA MCKESSON DRUG CO.,3000 KENSKILL AVE,WASHINGTON CT HOUSE,OH,43160,FAYETTE,...,OXYCODONE,TAB,1.50,10.0,2011-03-10,"Par Pharmaceutical, Inc.",McKesson Corporation,3.58600,400.0,5379.000


In [5]:
# insepct the columns
sample.columns

Index(['REPORTER_DEA_NO', 'REPORTER_BUS_ACT', 'REPORTER_NAME',
       'REPORTER_ADDL_CO_INFO', 'REPORTER_ADDRESS1', 'REPORTER_ADDRESS2',
       'REPORTER_CITY', 'REPORTER_STATE', 'REPORTER_ZIP', 'REPORTER_COUNTY',
       'BUYER_DEA_NO', 'BUYER_BUS_ACT', 'BUYER_NAME', 'BUYER_ADDL_CO_INFO',
       'BUYER_ADDRESS1', 'BUYER_ADDRESS2', 'BUYER_CITY', 'BUYER_STATE',
       'BUYER_ZIP', 'BUYER_COUNTY', 'TRANSACTION_CODE', 'DRUG_CODE', 'NDC_NO',
       'DRUG_NAME', 'Measure', 'MME_Conversion_Factor', 'Dosage_Strength',
       'TRANSACTION_DATE', 'Combined_Labeler_Name', 'Reporter_family',
       'CALC_BASE_WT_IN_GM', 'DOSAGE_UNIT', 'MME'],
      dtype='object')

In [ ]:
sample["DRUG_NAME"].unique()  # all contain opiates

array(['CODEINE', 'OXYCODONE', 'HYDROCODONE', 'METHADONE', 'FENTANYL',
       'BUPRENORPHINE', 'MORPHINE', 'OXYMORPHONE'], dtype=object)

In [ ]:
# listing only the needed columns and states
filepath = "/Users/brucechen/Documents/IDS720 PDS/pds-2025-opioids-study_on_opioids/arcos_all.zip"

selected_cols = [
    "BUYER_STATE",
    "BUYER_COUNTY",
    "TRANSACTION_DATE",
    "MME_Conversion_Factor",
    "CALC_BASE_WT_IN_GM",
]

selected_states = ["FL", "WA", "NC", "GA", "OR", "SC", "ID", "MT"]

pd.set_option("mode.copy_on_write", True)
chunks = []

In [ ]:
# process the data in chunks
for i, chunk in enumerate(
    pd.read_csv(
        filepath,
        chunksize=800_000,
        compression="zip",
        delimiter="\t",
        usecols=selected_cols,
        low_memory=True,
    )
):
    # filter for selected states
    state_selection = chunk[chunk["BUYER_STATE"].isin(selected_states)]
    state_selection["TOTAL_MME"] = (
        state_selection["MME_Conversion_Factor"] * state_selection["CALC_BASE_WT_IN_GM"]
    )

    # extract year from date
    state_selection["YEAR"] = pd.to_datetime(
        state_selection["TRANSACTION_DATE"]
    ).dt.year

    # group by state, county, year
    grouped = state_selection.groupby(
        ["BUYER_STATE", "BUYER_COUNTY", "YEAR"], as_index=False
    )["TOTAL_MME"].sum()

    chunks.append(grouped)
    print(f"Processed chunk {i+1}")

df = pd.concat(chunks, ignore_index=True)

# final aggregation since each chunk is grouped separately
df.groupby(["BUYER_STATE", "BUYER_COUNTY", "YEAR"], as_index=False)["TOTAL_MME"].sum()

df.sample(10)

Processed chunk 1
Processed chunk 2
Processed chunk 3
Processed chunk 4
Processed chunk 5
Processed chunk 6
Processed chunk 7
Processed chunk 8
Processed chunk 9
Processed chunk 10
Processed chunk 11
Processed chunk 12
Processed chunk 13
Processed chunk 14
Processed chunk 15
Processed chunk 16
Processed chunk 17
Processed chunk 18
Processed chunk 19
Processed chunk 20
Processed chunk 21
Processed chunk 22
Processed chunk 23
Processed chunk 24
Processed chunk 25
Processed chunk 26
Processed chunk 27
Processed chunk 28
Processed chunk 29
Processed chunk 30
Processed chunk 31
Processed chunk 32
Processed chunk 33
Processed chunk 34
Processed chunk 35
Processed chunk 36
Processed chunk 37
Processed chunk 38
Processed chunk 39
Processed chunk 40
Processed chunk 41
Processed chunk 42
Processed chunk 43
Processed chunk 44
Processed chunk 45
Processed chunk 46
Processed chunk 47
Processed chunk 48
Processed chunk 49
Processed chunk 50
Processed chunk 51
Processed chunk 52
Processed chunk 53
Pr

,BUYER_STATE,BUYER_COUNTY,YEAR,TOTAL_MME
39004,FL,WAKULLA,2012,598.042616
8440,NC,CABARRUS,2016,104649.146030
153808,WA,WHATCOM,2009,14215.932504
123133,NC,TRANSYLVANIA,2014,1725.908836
79620,GA,BARROW,2009,2947.851977
24847,NC,CRAVEN,2015,22239.008329
40125,FL,SAINT LUCIE,2014,8116.942074
140038,NC,PERSON,2007,596.380355
144071,WA,FERRY,2008,458.224193
154898,SC,MCCORMICK,2014,290.503030


In [ ]:
# export to Parquet file in the same directory as the notebook
import os

# Example new file name
new_filename = "arcos_processed.parquet"

# Get current notebook directory
notebook_dir = os.getcwd()

# Create full path with new filename
parquet_path = os.path.join(notebook_dir, new_filename)

# Save DataFrame to the new Parquet file
df.to_parquet(parquet_path, index=False)

print(f"DataFrame saved to {parquet_path}")

DataFrame saved to /Users/brucechen/Documents/IDS720 PDS/pds-2025-opioids-study_on_opioids/arcos_processed.parquet
